In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import glob
from ipywidgets import widgets
from ipywidgets import Button, HBox, VBox, Text, interactive_output

In [46]:
markers = ['o']
normalization_options = ['Specific Capacitance (F/g)', 'Specific Capacity (mAh/g)', 'Specific Charge (C/g)', 
                         'Volumetric Capacitance (F/cm$^3$)', 'Volumetric Capacity (mAh/cm$^3$)',
                         'Volumetric Charge (C/cm$^3$)', 'Areal Capacitance (F/cm$^2$)', 
                         'Areal Capacity (mAh/cm$^2$)', 'Areal Charge (C/cm$^2$)']

# Reading in files/pretreating data

In [41]:
# Enter path to folder containing your data in quotes followed by /*. Should match "your_path/*"
file_list = glob.glob('E:/2022_03_11/text/*')
file_list.sort()

In [42]:
# Enter your scan rates as a comma separated list in the square brackets, e.g., [5, 10, 20, 50, 100]
scan_rates = [0.1, 0.5, 1.0, 2.0, 5.0]

In [89]:
# Replace 'None' with your values. Okay to leave as 'None' if you don't have a value.
electrode_mass = 1.49 #in mg
electrode_area = 1.13 #in cm^2
electrode_volume = 0.7 #in cm^3

# BioLogic rate performance

In [102]:
def rate_performance(file_list):
    '''
    Calculates average capacitance, capacity, and charge (and error values) 
    for a series of scan rates.Returns a plot of average capacitance with 
    error bars versus scan rate.
    '''
    potential_window = 0
    
    charge = np.empty(len(scan_rates))
    charge_err = np.empty(len(scan_rates))
    
    discharge = np.empty(len(scan_rates))
    discharge_err = np.empty(len(scan_rates))
    
    ce = np.empty(len(scan_rates))
    ce_err = np.empty(len(scan_rates))
    
    for idx, file in enumerate(file_list):
        with open(file, 'r') as f:
            data = pd.read_table(f)
            
            if potential_window == 0:
                potential_window += (max(data['Ewe/V']) - min(data['Ewe/V']))

            cycle_number = data['cycle number'].unique()
  
            charge_list = []
            charge_data = data[data['<I>/mA'] > 0]
            for n in cycle_number[1:-1]:
                charge_list.append(
                    np.trapz(
                    charge_data['<I>/mA'][charge_data['cycle number'] == n], 
                    charge_data['time/s'][charge_data['cycle number'] == n]
                    )
                )
                
            charge[idx] = np.mean(charge_list)
            charge_err[idx] = np.std(charge_list)

            discharge_list = []
            discharge_data = data[data['<I>/mA'] < 0]
            for n in cycle_number[1:-1]:
                discharge_list.append(
                    np.trapz(
                    discharge_data['<I>/mA'][discharge_data['cycle number'] == n], 
                    discharge_data['time/s'][discharge_data['cycle number'] == n]
                    )
                )
                
            discharge[idx] = np.abs(np.mean(discharge_list))
            discharge_err[idx] = np.abs(np.std(discharge_list))
            
            ce[idx] = np.mean([i / j for i, j in zip(np.abs(discharge_list), charge_list)])
            ce_err[idx] = np.std([i / j for i, j in zip(np.abs(discharge_list), charge_list)])
            
    return charge, discharge, charge_err, discharge_err, ce, ce_err, potential_window

In [103]:
charge, discharge, charge_err, discharge_err, ce, ce_err, potential_window = rate_performance(file_list=file_list)

In [157]:
def plot_rate_perf(yaxis, markers, c_e=False):
    
    global fig
    
    fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
        
    if 'Capacitance' in yaxis:
        capacitance = charge / potential_window
        capacitance_err = charge_err / potential_window
        dis_capacitance = discharge / potential_window
        dis_capacitance_err = discharge_err / potential_window
        
        if 'Specific' in yaxis:
            capacitance = capacitance / electrode_mass
            capacitance_err = capacitance_err / electrode_mass
            dis_capacitance = dis_capacitance / electrode_mass
            dis_capacitance_err = dis_capacitance_err / electrode_mass
        
        elif 'Volumetric' in yaxis:
            capacitance = capacitance / (electrode_volume * 1000)
            capacitance_err = capacitance_err / (electrode_volume * 1000)
            dis_capacitance = dis_capacitance / (electrode_volume * 1000)
            dis_capacitance_err = dis_capacitance_err / (electrode_volume * 1000)
            
        elif 'Areal' in yaxis:
            capacitance = capacitance / (electrode_area * 1000)
            capacitance_err = capacitance_err / (electrode_area * 1000)
            dis_capacitance = dis_capacitance / (electrode_area * 1000)
            dis_capacitance_err = dis_capacitance_err / (electrode_area * 1000)
            
        ax.errorbar(scan_rates, capacitance, yerr=capacitance_err,
                    label='Charge',
                    marker=markers, 
                    markersize=12,
                    color='red', 
                    capsize=5,
                    capthick=2
                   )
    
        ax.errorbar(scan_rates, dis_capacitance, yerr=dis_capacitance_err,
                    label='Discharge',
                    marker=markers,
                    markersize=12,
                    color='blue',
                    capsize=5,
                    capthick=2
                   )            
    
    elif "Capacity" in yaxis:
        capacity = charge / 3600
        capacity_err = charge_err / 3600
        dis_capacity = discharge / 3600
        dis_capacity_err = discharge_err / 3600
        
        if 'Specific' in yaxis:
            capacity = (capacity * 1000) / electrode_mass
            capacity_err = (capacity_err * 1000) / electrode_mass
            dis_capacity = (dis_capacity * 1000) / electrode_mass
            dis_capacity_err = (dis_capacity_err * 1000) / electrode_mass
        
        elif 'Volumetric' in yaxis:
            capacity = capacity / electrode_volume
            capacity_err = capacity_err / electrode_volume
            dis_capacity = dis_capacity / electrode_volume
            dis_capacity_err = dis_capacity_err / electrode_volume
        
        elif 'Areal' in yaxis:
            capacity = capacity / electrode_area
            capacity_err = capacity_err / electrode_area
            dis_capacity = dis_capacity / electrode_area
            dis_capacity_err = dis_capacity_err / electrode_area
            
        ax.errorbar(scan_rates, capacity, yerr=capacity_err,
                    label='Charge',
                    marker=markers, 
                    markersize=12,
                    color='red', 
                    capsize=5,
                    capthick=2
                           )
    
        ax.errorbar(scan_rates, dis_capacity, yerr=dis_capacity_err,
                    label='Discharge',
                    marker=markers,
                    markersize=12,
                    color='blue',
                    capsize=5,
                    capthick=2
                   ) 
            
    elif 'Charge' in yaxis:
        coulombs = charge / 1000
        coulombs_err = charge_err / 1000
        dis_coulombs = discharge / 1000
        dis_coulombs_err = discharge_err / 1000
        
        if 'Specific' in yaxis:
            coulombs = (coulombs * 1000) / electrode_mass
            coulombs_err = (coulombs_err * 1000) / electrode_mass
            dis_coulombs = (dis_coulombs * 1000) / electrode_mass
            dis_coulombs_err = (dis_coulombs_err * 1000) / electrode_mass
        
        elif 'Volumetric' in yaxis:
            coulombs = coulombs / electrode_volume
            coulombs_err = coulombs_err / electrode_volume
            dis_coulombs = dis_coulombs / electrode_volume
            dis_coulombs_err = dis_coulombs_err / electrode_volume
        
        elif 'Areal' in yaxis:
            coulombs = coulombs / electrode_area
            coulombs_err = coulombs_err / electrode_area
            dis_coulombs = dis_coulombs / electrode_area
            dis_coulombs_err = dis_coulombs_err / electrode_area
            
        ax.errorbar(scan_rates, coulombs, yerr=coulombs_err,
                    label='Charge',
                    marker=markers, 
                    markersize=12,
                    color='red', 
                    capsize=5,
                    capthick=2
                           )
    
        ax.errorbar(scan_rates, dis_coulombs, yerr=dis_coulombs_err,
                    label='Discharge',
                    marker=markers,
                    markersize=12,
                    color='blue',
                    capsize=5,
                    capthick=2
                   ) 
   


    if c_e:
        ax2 = ax.twinx()
        ax2.errorbar(scan_rates, ce * 100, yerr=ce_err * 100,
                label='Charge',
                marker=markers, 
                markersize=12,
                color='red', 
                capsize=5,
                capthick=2
                       )

    ax.set_xlabel('Scan rate (mV/s)', fontsize=24)
    ax.set_ylabel(yaxis, fontsize=24)
    ax.tick_params(which='both', labelsize=18, width=2, length=5)
    ax.set_xscale('log')
#     ax.set_ylim(0, max(discharge) * 1.1)
    ax.xaxis.set_major_formatter(mticker.ScalarFormatter())
    ax.xaxis.get_major_formatter().set_scientific(False)

In [151]:
def make_figure():
    '''
    Generates interactive figure using the plot_CVs_biologic function.
    '''
    yaxis_choice = widgets.Dropdown(options=normalization_options)
    marker_choice = widgets.Dropdown(options=markers)
    c_e = widgets.Checkbox(description='Coulombic Efficiency', indent=False)
    def on_button_clicked(b):
        fig.savefig("test.png")

    button = Button(description="Save Figure")
    button.on_click(on_button_clicked)

    ui = HBox([yaxis_choice, marker_choice, c_e, button])

    out = widgets.interactive_output(plot_rate_perf, {'yaxis': yaxis_choice, 'markers': marker_choice, 'c_e': c_e})
    display(ui, out)

In [158]:
make_figure()

Output()